In [3]:
import numpy as np
import scipy as sp
import scipy.signal
import matplotlib.pyplot as plt
import torch
import torchvision.models as models
from torchvision import transforms as T
import sys
import os
from PIL import Image, ImageFilter 
from shutil import copyfile
# import elasticdeform
import cv2 as cv

In [8]:
def sample_warp_field(im, warp_field):
    """
    creates an image outim by assigning each pixel outim[X] of outim the value of im[warp_field[X]].
    since X is floating point, we need to do bilinear interpolation
    """
    inds = warp_field.copy();
    inds[:,:,0]=inds[:,:,0]*im.shape[0];
    inds[:,:,1]=inds[:,:,1]*im.shape[1];

    indsxf = np.clip(
        np.floor(inds[:,:,1]),
        0,
        im.shape[1]-1
    ).astype(np.int);
    
    indsxc = np.clip(
        np.ceil(inds[:,:,1]),
        0,
        im.shape[1]-1
    ).astype(np.int);
    
    indsyf = np.clip(
        np.floor(inds[:,:,0]),
        0,
        im.shape[0]-1
    ).astype(np.int);
    
    indsyc = np.clip(
        np.ceil(inds[:,:,0]),
        0,
        im.shape[0]-1
    ).astype(np.int);

    indsx = inds[:,:,1]-indsxf;
    indsy = inds[:,:,0]-indsyf;
    
    indsx = np.dstack((indsx,indsx,indsx));
    indsy = np.dstack((indsy,indsy,indsy));

    opixa = (1.0-indsx)*im[indsyf,indsxf]+indsx*im[indsyf,indsxc];
    opixb = (1.0-indsx)*im[indsyc,indsxf]+indsx*im[indsyc,indsxc];
    opix = opixb*indsy+(1.0-indsy)*opixa;
    return opix



def computeWarpField(size, warp_sample_size=None, noise_scale=None, warp_probability=1.0, fftconvolve=True, rng=None):
    """
    Generates warp field
    output is [height, width,2]
    each value [y,x] is the normalized coordinate that you will sample from to get the warped image
    these are floating point, so we will need to interpolate
    
    warp_probability is the probability of warping each vertex. So each vertex is randomly either warped or not.
    fftconvolve will make things WAY faster if you have a small warp_sample_size compared to size
    rng is an optional random number generator, which you can seed if you like
    """
    if(rng is None):
        rng = np.random.default_rng();
    
    sparse = np.zeros(size);
    if(noise_scale is None):
        noise_scale = 0.5;
    if(warp_sample_size is None):
        warp_sample_size = np.array([int(size[0]*0.1),int(size[1]*0.1), 2]);
        
    X = np.linspace(0,1,warp_sample_size[1])
    Y = np.linspace(0,1,warp_sample_size[0])    
    xx,yy = np.meshgrid(X,Y)
    start_samples = np.dstack((yy,xx));
    warp_samples = start_samples.copy();
    x_range = X[1]*2*noise_scale;
    y_range = Y[1]*2*noise_scale;
    
    y_warp_amount = y_range*(rng.random(warp_sample_size[:2])-0.5);
    x_warp_amount = x_range*(rng.random(warp_sample_size[:2])-0.5);
    
    switch_displacements = rng.random(warp_sample_size[:2])>warp_probability;
    y_warp_amount[switch_displacements]=0;
    x_warp_amount[switch_displacements]=0;
    
    
#     warp_samples[:,:,0]=warp_samples[:,:,0]+y_warp_amount;
#     warp_samples[:,:,1]=warp_samples[:,:,1]+x_warp_amount;
    warp_samples[:,:,0]=y_warp_amount;
    warp_samples[:,:,1]=x_warp_amount;

    windowx = int(3*size[1]/warp_sample_size[1]);
    stdx = size[1]/(2*warp_sample_size[1]);
    windowy = int(3*size[0]/warp_sample_size[0]);
    stdy = size[0]/(2*warp_sample_size[0]);
    
#     def kst(wind):
#         return np.sqrt(((wind**2)-1)/12);
    
    sigy = sp.signal.windows.gaussian(windowy, std=stdy);
    sigx = sp.signal.windows.gaussian(windowx, std=stdx);
#     sigy = sp.signal.windows.gaussian(windowy, std=kst(windowy));
#     sigx = sp.signal.windows.gaussian(windowx, std=kst(windowx));
    kernel = np.outer(sigy, sigx);
    kernel = kernel/np.sum(kernel);
#     print('here',windowx, windowy, stdx, stdy, warp_sample_size, size)
    start_sample_inds = start_samples.copy();
    start_sample_inds[:,:,0]=np.clip(start_sample_inds[:,:,0]*size[0],0,size[0]-1);
    start_sample_inds[:,:,1]=np.clip(start_sample_inds[:,:,1]*size[1],0,size[1]-1);
    start_sample_inds = start_sample_inds.astype(np.int)


    coords = np.zeros([size[0],size[1],3]);
    coords[start_sample_inds[:,:,0],start_sample_inds[:,:,1],:2]=warp_samples
    coords[start_sample_inds[:,:,0],start_sample_inds[:,:,1],2]=1

    if(fftconvolve):
        for c in range(3):
            coords[:,:,c] = sp.signal.fftconvolve(coords[:,:,c], kernel, mode='same');
    else:
        for c in range(3):
            coords[:,:,c] = sp.signal.convolve(coords[:,:,c], kernel);
    
    cX = np.linspace(0,1,size[1])
    cY = np.linspace(0,1,size[0])    
    cxx,cyy = np.meshgrid(cX,cY)
    r_samples = np.dstack((cyy,cxx));
        

    coords[:,:,0]=coords[:,:,0]/coords[:,:,2];
    coords[:,:,1]=coords[:,:,1]/coords[:,:,2];
    coords[:,:,:2]= coords[:,:,:2]+r_samples;

    return coords[:,:,:2]
            
            
def elastic_deform(bg, grid=10, distortion_range=[0.2, 1], each_grid_prob=0.5):
    """
    positive example augmentation: simulate elastic deformation
    bg: np array, rgb
    """
#     print('grid', grid)
    rng = np.random.default_rng();
    ntapsv = grid
    n = np.random.uniform(0.2, 1)
    aspect = bg.shape[1]/bg.shape[0]
    warp_field = computeWarpField(
            bg.shape, 
            warp_sample_size=[int(ntapsv),int(ntapsv*aspect)],
            noise_scale=n,
            rng=rng,
            warp_probability=each_grid_prob
        )
    warped = sample_warp_field(bg, warp_field);
    return warped.astype('uint8')


def darken_uniform(img_np, darken_range=[-10, 30]):
    """
    positive example augmentation: darkening the whole scene
    img_np: np array
    add_uniform, add_gaussian: probability of using each mean
    """
    darkened_uniform = img_np.astype('int') - np.random.randint(darken_range[0], high=darken_range[1])
    darkened_uniform_clipped = np.clip(darkened_uniform, 0, 255)
    return darkened_uniform_clipped.astype('uint8')


def darken_gaussian(img_np, mesh_X, mesh_Y, mag_range=[0,100], sig_range=[20,50], boundaries=None): 
    """
    positive example augmentation: add a gaussian darkness to img_np.
    """
    if boundaries is not None:
        left, right, up, down = boundaries
    else:
         left, right, up, down = 0, img_np.shape[1], 0, img_np.shape[0]
    sigma = np.random.randint(sig_range[0], high=sig_range[1]) #was 100 for pure warp
    mu_X = np.random.randint(left, right)
    mu_Y = np.random.randint(up, down)
    mag = np.random.randint(mag_range[0], high=mag_range[1])
    print('gaussian', mag, mu_X, mu_Y)
    gaussian = (mag * np.exp(-((mesh_X - mu_X)**2 + (mesh_Y - mu_Y)**2)/(2.0 * sigma ** 2))).astype('int')
    darkened_gaussian = img_np.astype('int') - np.moveaxis(np.tile(gaussian, (3, 1, 1)), 0, -1)
    darkened_gaussian_clipped = np.clip(darkened_gaussian, 0, 255)
    return darkened_gaussian_clipped.astype('uint8')

    
def homo_deform(bg, boundaries):
    """
    positive example augmentation: deform the bg image 
    within the boundaries by a random homography.
    
    bg: np array
    """
    scale = 0.2
    left, right, up, down = boundaries
    src_pts = np.array([[left,up],[right,up], [right,down], [left,down]])
    h, w, _ = bg.shape
    left1 = left + np.random.uniform(-scale*w, scale*w)
    left2 = left + np.random.uniform(-scale*w, scale*w)
    right1 = right + np.random.uniform(-scale*w, scale*w)
    right2 = right + np.random.uniform(-scale*w, scale*w)
    up1 = up + np.random.uniform(-scale*h, scale*h)
    up2 = up + np.random.uniform(-scale*h, scale*h)
    down1 = down + np.random.uniform(-scale*h, scale*h)
    down2 = down + np.random.uniform(-scale*h, scale*h)

    dst_pts = np.array([[left1, up1], [right1, up2], [right2, down1], [left2, down2]])
    M, _ = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.0)
    out = cv.warpPerspective(bg, M, (w, h), flags=cv.INTER_LINEAR) #[up:down, left:right]
    return out, dst_pts

def homo_deform_new(bg, boundaries, scale=0.1):
    """
    bg: np array
    """
    left, right, up, down = boundaries
    src_pts = np.array([[left,up],[right,up], [right,down], [left,down]])
    h, w, _ = bg.shape
    left1 = left + np.random.uniform(-scale*w, 0)
    left2 = left + np.random.uniform(-scale*w, 0)
    right1 = right + np.random.uniform(0, scale*w)
    right2 = right + np.random.uniform(0, scale*w)
    up1 = up + np.random.uniform(-scale*h, 0)
    up2 = up + np.random.uniform(-scale*h, 0)
    down1 = down + np.random.uniform(0, scale*h)
    down2 = down + np.random.uniform(0, scale*h)
#     print(left1,up1, right1,up2, right2,down1, left2,down2)
    dst_pts = np.array([[left1, up1], [right1, up2], [right2, down1], [left2, down2]])
    M, _ = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.0)
    out = cv.warpPerspective(bg, M, (w, h), flags=cv.INTER_LINEAR) #[up:down, left:right]
    return out, dst_pts


def gen_pos_ex_bg(basedir, sourcedir, boundaries, add_elastic, add_homo, add_uniform, add_gaussian, \
                  add_blurr_or_noise, add_gaussian_noise, elastic_kwargs=None, uniform_darken_kwargs=None, \
                  gaussian_darken_kwargs=None, homo_kwargs=None, blur_kwargs=None, noise_kwargs=None, \
                  num=2500, folder_suffix=''):
    
    bgs = sorted([i for i in os.listdir(sourcedir) if ('png' in i) or ('jpg' in i)])
    gt = np.asarray(Image.open(os.path.join(sourcedir, bgs[0])).convert('RGB'))
#     gt = np.asarray(Image.open(os.path.join(sourcedir,'bg_gt.png')).convert('RGB'))
    x = np.arange(0, gt.shape[1])
    y = np.arange(0, gt.shape[0])
    [X,Y] = np.meshgrid(x, y) 

    [left, right, up, down] = boundaries 
    save_dir = os.path.join(basedir, 'dis_real_l1', '_'.join([str(add_uniform)+'uniform', \
                                                            str(add_gaussian)+'gaussian_dark', \
                                                              str(add_homo)+'flip', \
                                                              str(add_elastic)+'elastic',\
                                                             str((1-add_gaussian_noise)*add_blurr_or_noise)+'blursigma'+ \
                                                              str(blur_kwargs['sigma_range'][0]) + str(blur_kwargs['sigma_range'][0])+'k'+str(blur_kwargs['kernel']),\
                                                             str(add_gaussian_noise*add_blurr_or_noise)+'gaussian_noise_std'+\
                                                              str(noise_kwargs['std_range'][0])+str(noise_kwargs['std_range'][1])+'mean'+str(noise_kwargs['mean']),\
                                                             folder_suffix]))
    os.makedirs(save_dir)
    print('The positive examples will be saved to', save_dir)
    for n in range(num):
        # decision vector is like rolling one dice for every augmentation 
        # to decide whether to apply it for the current generation.
        decision = np.random.uniform(size=6)
        # randomly choose a background image to start with
#         rand_idx = np.random.randint(len(bgs))
        rand_idx = np.random.randint(250)
        print(n, rand_idx, decision)
#         out = gt.copy()
        out = np.asarray(Image.open(os.path.join(sourcedir, bgs[rand_idx])).convert('RGB'))
        if decision[0] < add_elastic:
            print('elastic')
            out = elastic_deform(out, **elastic_kwargs)
#         if decision[1] < add_homo:
#             print('flip')
#             out = flip(out)
        if decision[1] < add_homo:
            print('homo')
            out, dst = homo_deform_new(out, boundaries, **homo_kwargs)
        if decision[2] < add_uniform:
            print('uniform')
            out = darken_uniform(out, **uniform_darken_kwargs)
        if decision[3] < add_gaussian:
            print('gaussian')
            out = darken_gaussian(out, X, Y, boundaries=boundaries, **gaussian_darken_kwargs)
        
        if decision[4] < add_blurr_or_noise:
            if decision[5] < add_gaussian_noise:
                print('gaussian noise')
                out = add_noise(out, **noise_kwargs)
            # blur and noise are exclusive
            else:
                print('blur')
                out = add_blur(out, **blur_kwargs)
            
        canvas = Image.fromarray(out).convert('RGB')
        canvas.save(os.path.join(save_dir, str(n)+'.png'))


def add_blur(img, sigma_range=[0.2, 1], kernel=5):
    img = img.astype("int16")
    std = np.random.uniform(sigma_range[0], high=sigma_range[1])
    blur_img = cv.GaussianBlur(img, (kernel, kernel), std, borderType = cv.BORDER_REPLICATE)
    blur_img = ceil_floor_image(blur_img)
    return blur_img

def ceil_floor_image(image):
    """
    Args:
        image : numpy array of image in datatype int16
    Return :
        image : numpy array of image in datatype uint8 with ceilling(maximum 255) and flooring(minimum 0)
    """
    image[image > 255] = 255
    image[image < 0] = 0
    image = image.astype("uint8")
    return image

def add_noise(img, std_range=[0, 20], mean=0):
    std = np.random.randint(std_range[0], high=std_range[1])
    print('std', std)
    gaussian_noise = np.random.normal(mean, std, img.shape)
    img = img.astype("int16")
    noise_img = img + gaussian_noise
    noise_img = ceil_floor_image(noise_img)
    return noise_img

In [10]:
from gen_foregroundPosEx import rotate, flip
# the save output directory
datadir = '../datasets/composited/half_body/dancing_woman_half'
# the directory from which the residual marginal priors are used for the following augmentations
sourcedir = '../../TCVOM/VM108/BG_done/BG_done2/lake'
# the corners of the images, should be height and width
left = 0
right = 512
up = 0
down = 352
boundaries  = [left, right, up, down] 
# The probability of applying each augmentation during the generation of each positive example
elastic_prob = 0.25
homo_prob = 0.5
darken_uniform_prob = 0.5
darken_gaussian_prob = 0.5
blur_or_gaussian_prob = 0.5
gaussian_noise_prob = 0.5

elastic_kwargs={
    'grid': 10, 
    'distortion_range': [0.2, 1], 
    'each_grid_prob': 0.5
}
uniform_darken_kwargs={
    'darken_range':[-30, 30]
}
gaussian_darken_kwargs={
    'mag_range': [0,50], 
    'sig_range': [20,50]
}
homo_kwargs={
    'scale': 0.1
}
blur_kwargs= {'sigma_range':[0.2, 1], 'kernel': 5}
gaussian_noise_kwargs= {'std_range':[2, 7], 'mean':0}
# rawframes if from different rgba0, none if from a single bg
gen_pos_ex_bg(datadir, sourcedir, boundaries, elastic_prob, homo_prob, darken_uniform_prob, darken_gaussian_prob, \
              blur_or_gaussian_prob, gaussian_noise_prob, elastic_kwargs=elastic_kwargs, \
              uniform_darken_kwargs=uniform_darken_kwargs, \
              gaussian_darken_kwargs=gaussian_darken_kwargs, homo_kwargs=homo_kwargs,\
              blur_kwargs=blur_kwargs, noise_kwargs=gaussian_noise_kwargs,
              num=1200, folder_suffix='')
        

The positive examples will be saved to ../datasets/composited/half_body/dancing_woman_half/dis_real_l1/0.5uniform_0.5gaussian_dark_0.5flip_0elastic_0.25blursigma0.20.2k5_0.25gaussian_noise_std27mean0_gtbg
0 35 [0.86914237 0.46730018 0.43333836 0.04563867 0.83551772 0.0688707 ]
homo
uniform
gaussian
gaussian 22 6 201
1 188 [0.11639718 0.51029845 0.98195646 0.45665855 0.11043167 0.2174791 ]
gaussian
gaussian 1 48 153
gaussian noise
std 6
2 94 [0.24882767 0.22173018 0.67082625 0.77409315 0.67215289 0.57815941]
homo
3 108 [0.63438256 0.90065383 0.48566864 0.80260914 0.16910261 0.57957354]
uniform
blur
4 137 [0.29142942 0.40400603 0.58154493 0.26612746 0.5395861  0.81664052]
homo
gaussian
gaussian 46 406 96
5 44 [0.7736471  0.78594842 0.32776072 0.44487749 0.8493597  0.59022093]
uniform
gaussian
gaussian 38 179 116
6 74 [0.01571217 0.10252974 0.81807688 0.55170421 0.83931527 0.56330579]
homo
7 92 [0.11022068 0.0026522  0.01401514 0.33854332 0.10181265 0.72583985]
homo
uniform
gaussian
gauss

81 88 [0.48775553 0.69454624 0.99515167 0.39773619 0.71158801 0.83324666]
gaussian
gaussian 46 86 52
82 142 [0.10879016 0.40162886 0.445722   0.24767551 0.42692397 0.63011304]
homo
uniform
gaussian
gaussian 40 410 10
blur
83 157 [0.48538628 0.3506677  0.31457922 0.58013778 0.42573768 0.13508755]
homo
uniform
gaussian noise
std 5
84 147 [0.11345174 0.5867488  0.05968726 0.01430499 0.03620923 0.1557304 ]
uniform
gaussian
gaussian 26 162 87
gaussian noise
std 3
85 71 [0.03706988 0.59115283 0.25626184 0.19189331 0.73941926 0.85219756]
uniform
gaussian
gaussian 42 221 340
86 112 [0.26937397 0.74099778 0.53796599 0.86773014 0.40848641 0.21779885]
gaussian noise
std 2
87 22 [0.77061724 0.99108584 0.94819762 0.93683761 0.13911734 0.30569557]
gaussian noise
std 6
88 244 [0.23826352 0.77309647 0.87315129 0.98832078 0.13093583 0.40211171]
gaussian noise
std 5
89 159 [0.75439839 0.29352771 0.36195451 0.08827612 0.01474949 0.63439441]
homo
uniform
gaussian
gaussian 12 221 25
blur
90 104 [0.62539442

161 233 [0.57174345 0.05244489 0.12739508 0.10933394 0.1899812  0.09061063]
homo
uniform
gaussian
gaussian 36 441 119
gaussian noise
std 2
162 8 [0.84871947 0.02004384 0.22391301 0.35749075 0.27849877 0.49993763]
homo
uniform
gaussian
gaussian 45 300 281
gaussian noise
std 3
163 30 [0.01655346 0.68717267 0.89797378 0.19941388 0.2541782  0.80116504]
gaussian
gaussian 2 322 82
blur
164 132 [0.44880679 0.77849464 0.66696715 0.00703394 0.37321387 0.28531781]
gaussian
gaussian 9 465 283
gaussian noise
std 4
165 34 [0.67159725 0.97762811 0.40760492 0.67257063 0.75550829 0.97609653]
uniform
166 166 [0.13885977 0.56505154 0.18625361 0.77648501 0.71958366 0.61132187]
uniform
167 197 [0.91454926 0.84863104 0.91434294 0.51342095 0.61081809 0.56203154]
168 53 [0.07050879 0.82182807 0.04089945 0.33984982 0.72474499 0.04418065]
uniform
gaussian
gaussian 11 396 336
169 128 [0.41248362 0.3315013  0.48144246 0.75001603 0.74115829 0.26248207]
homo
uniform
170 18 [0.02339403 0.92736215 0.81317013 0.16656

blur
242 88 [0.37324585 0.61092948 0.42392301 0.00785444 0.4959541  0.82554056]
uniform
gaussian
gaussian 14 407 266
blur
243 53 [0.08548511 0.46457115 0.06822804 0.38391968 0.51826503 0.9905085 ]
homo
uniform
gaussian
gaussian 29 55 91
244 239 [0.70634355 0.48212823 0.80340855 0.01274596 0.95513041 0.5576802 ]
homo
gaussian
gaussian 37 238 103
245 137 [0.38418402 0.30014819 0.80691065 0.71461803 0.54881105 0.07464119]
homo
246 7 [0.27146839 0.69416353 0.98021973 0.48613393 0.59290411 0.89851895]
gaussian
gaussian 33 453 137
247 160 [0.15466661 0.3150885  0.97142438 0.54784432 0.47375648 0.29403069]
homo
gaussian noise
std 4
248 194 [0.86138353 0.74830615 0.63217025 0.92389193 0.48264206 0.35126263]
gaussian noise
std 2
249 172 [0.53626019 0.76662121 0.92784093 0.67006823 0.57428067 0.67337363]
250 161 [0.09470674 0.46817125 0.860298   0.6116155  0.19932154 0.66107266]
homo
blur
251 15 [0.17278171 0.28886326 0.34562878 0.87883324 0.54763591 0.97480888]
homo
uniform
252 143 [0.07545011 

324 236 [0.35018472 0.37839611 0.0038628  0.25016662 0.37567135 0.6576283 ]
homo
uniform
gaussian
gaussian 30 0 270
blur
325 82 [0.90443934 0.26751865 0.57581818 0.16753406 0.00727165 0.3016021 ]
homo
gaussian
gaussian 24 388 327
gaussian noise
std 3
326 11 [0.23537176 0.69708483 0.81023987 0.39262242 0.56511829 0.92947537]
gaussian
gaussian 22 409 10
327 52 [0.19874396 0.14395754 0.74501379 0.66863142 0.74006853 0.3394949 ]
homo
328 68 [0.57623417 0.88371912 0.74809387 0.63746245 0.16420679 0.01936355]
gaussian noise
std 2
329 68 [0.02574477 0.15155407 0.60575627 0.08125244 0.66281518 0.07353978]
homo
gaussian
gaussian 15 167 238
330 157 [0.01099786 0.98407161 0.37129566 0.41885103 0.38581743 0.03728393]
uniform
gaussian
gaussian 22 281 309
gaussian noise
std 6
331 115 [0.98876913 0.3259718  0.00868165 0.87044353 0.90358726 0.27215023]
homo
uniform
332 105 [0.48555412 0.30080601 0.34276737 0.29671433 0.34620502 0.27275758]
homo
uniform
gaussian
gaussian 38 42 241
gaussian noise
std 2


...

802 110 [0.74318174 0.42249401 0.29868814 0.58206396 0.566843   0.74346609]
homo
uniform
803 84 [0.00194127 0.54933321 0.42608278 0.69282692 0.52205748 0.58979914]
uniform
804 161 [0.13579342 0.56207554 0.20438824 0.43874301 0.41441568 0.05381796]
uniform
gaussian
gaussian 42 20 341
gaussian noise
std 6
805 214 [0.8136446  0.76960549 0.59684997 0.02672068 0.29976409 0.00700791]
gaussian
gaussian 36 284 322
gaussian noise
std 5
806 116 [0.84404791 0.69915926 0.65945781 0.11194145 0.38287844 0.25209762]
gaussian
gaussian 46 508 235
gaussian noise
std 4
807 194 [0.2359704  0.39785387 0.71319396 0.20765878 0.87112227 0.11118019]
homo
gaussian
gaussian 2 123 251
808 36 [0.8202279  0.78719872 0.9115422  0.06123848 0.34895946 0.29467818]
gaussian
gaussian 20 178 102
gaussian noise
std 2
809 105 [0.20520886 0.84366163 0.15933357 0.57333199 0.01682689 0.82935951]
uniform
blur
810 158 [0.92827354 0.71142503 0.49301508 0.56918486 0.78970352 0.87973754]
uniform
811 100 [0.18114701 0.88325318 0.956

882 200 [0.97568698 0.2578706  0.29169079 0.89874288 0.44367239 0.74387292]
homo
uniform
blur
883 76 [0.49998387 0.11106528 0.49343272 0.86481    0.89441364 0.97156322]
homo
uniform
884 60 [0.81875767 0.45665578 0.57377018 0.84129009 0.22885451 0.44250302]
homo
gaussian noise
std 2
885 182 [0.85301924 0.85173943 0.65327012 0.86083316 0.26906543 0.22709874]
gaussian noise
std 4
886 240 [0.2312917  0.26738811 0.72191264 0.50927964 0.57069745 0.79891354]
homo
887 142 [0.62610719 0.09122219 0.76527325 0.81418092 0.39033353 0.99028114]
homo
blur
888 56 [0.21958346 0.86348404 0.49837206 0.01505695 0.94716196 0.62579375]
uniform
gaussian
gaussian 48 393 131
889 150 [0.59778006 0.61041915 0.52781825 0.07766294 0.48493668 0.78810392]
gaussian
gaussian 31 99 18
blur
890 123 [0.57510258 0.85313828 0.44278862 0.49215049 0.27284844 0.11754098]
uniform
gaussian
gaussian 21 139 134
gaussian noise
std 5
891 51 [0.89460818 0.4462939  0.3912638  0.92148769 0.66437126 0.65172767]
homo
uniform
892 84 [0.1

961 211 [0.21883741 0.85331853 0.07429744 0.55012233 0.55155718 0.9482752 ]
uniform
962 207 [0.51807901 0.38985355 0.12032736 0.712283   0.57494849 0.24416579]
homo
uniform
963 94 [0.02416434 0.89968375 0.70366972 0.18281247 0.15464835 0.96947822]
gaussian
gaussian 39 312 194
blur
964 230 [0.17189586 0.1245836  0.89350728 0.23810095 0.73733996 0.90698904]
homo
gaussian
gaussian 13 57 85
965 96 [0.12574006 0.05160077 0.65176729 0.79983455 0.65412411 0.29536162]
homo
966 146 [0.15162651 0.50240178 0.34309813 0.19666765 0.17868798 0.71561056]
uniform
gaussian
gaussian 38 36 217
blur
967 227 [0.06975749 0.2215139  0.8647828  0.50345646 0.24321326 0.36393774]
homo
gaussian noise
std 3
968 211 [0.12973611 0.28648636 0.84805842 0.3135694  0.48936881 0.71942945]
homo
gaussian
gaussian 48 409 73
blur
969 39 [0.90877176 0.10893794 0.41104319 0.00171824 0.07083412 0.16682863]
homo
uniform
gaussian
gaussian 17 18 131
gaussian noise
std 6
970 234 [0.02262133 0.0260041  0.50568274 0.87898149 0.81647

1041 0 [0.78601295 0.57372612 0.47551289 0.58883996 0.37538571 0.99421442]
uniform
blur
1042 26 [0.58615444 0.17746442 0.20367842 0.49621463 0.33771674 0.30649503]
homo
uniform
gaussian
gaussian 17 227 158
gaussian noise
std 5
1043 6 [0.81343718 0.86138393 0.80552022 0.45974985 0.20447605 0.75433463]
gaussian
gaussian 28 45 114
blur
1044 177 [0.82063318 0.86660997 0.82854448 0.18717429 0.36203254 0.54725299]
gaussian
gaussian 11 65 278
blur
1045 227 [0.71283127 0.59972944 0.69220769 0.05926299 0.50709001 0.40008446]
gaussian
gaussian 14 289 71
1046 35 [0.67307617 0.16026598 0.23677525 0.26566215 0.30791039 0.96595922]
homo
uniform
gaussian
gaussian 12 382 328
blur
1047 34 [0.44967862 0.75420513 0.64577635 0.8404566  0.32700199 0.74746182]
blur
1048 28 [0.27460496 0.94919853 0.45939766 0.53932653 0.98701895 0.60321778]
uniform
1049 214 [0.74620334 0.872256   0.22209301 0.72410504 0.51464892 0.44161312]
uniform
1050 32 [0.55449094 0.47406019 0.7492704  0.61198004 0.2779371  0.82343579]
h

1121 239 [0.44599702 0.68025623 0.12191516 0.7156003  0.37373275 0.48504473]
uniform
gaussian noise
std 4
1122 175 [0.74600229 0.41588948 0.3164402  0.00843233 0.80010508 0.57007074]
homo
uniform
gaussian
gaussian 32 471 204
1123 23 [0.48499233 0.77517611 0.45078153 0.63141535 0.76787067 0.24306795]
uniform
1124 64 [0.08675882 0.24580453 0.55722628 0.12037607 0.42616811 0.95971797]
homo
gaussian
gaussian 12 28 119
blur
1125 160 [0.79759675 0.42108596 0.6305571  0.48937238 0.14555603 0.86900925]
homo
gaussian
gaussian 43 76 260
blur
1126 130 [0.05755187 0.37485591 0.28655717 0.04904693 0.89923871 0.01882438]
homo
uniform
gaussian
gaussian 15 503 266
1127 79 [0.19895419 0.15891672 0.91074968 0.33411457 0.83211792 0.19219523]
homo
gaussian
gaussian 3 327 0
1128 199 [0.35408335 0.78966017 0.72212971 0.33406193 0.14365448 0.85596731]
gaussian
gaussian 16 208 101
blur
1129 234 [0.14704736 0.69472222 0.19519523 0.02282445 0.93100753 0.36967048]
uniform
gaussian
gaussian 37 351 0
1130 229 [0.4